<h1><font color='#252526'>Predição de preços no setor imobiliário</font></h1>

<hr>

<h2><font color='#734838'>Sobre</font></h2>

<p>
A House Rocket é uma plataforma digital que tem como modelo de negócio, a compra e a venda de imóveis usando tecnologia.

Você é um Data Scientist contrato pela empresa para ajudar a encontrar as melhores oportunidades de negócio no mercado de imóveis. O CEO da House Rocket gostaria de maximizar a receita da empresa encontrando boas oportunidades de negócio.

Sua principal estratégia é comprar boas casas em ótimas localizações com preços baixos e depois revendê-las posteriormente à preços mais altos. Quanto maior a diferença entre a compra e a venda, maior o lucro da empresa e portanto maior sua receita.

Entretanto, as casas possuem muitos atributos que as tornam mais ou menos atrativas aos compradores e vendedores e a localização e o período do ano também podem influenciar os preços.
    
- Quais casas o CEO da House Rocket deveria comprar e por qual preço de compra?
- Uma vez a casa em posse da empresa, qual o melhor momento para vendê-las e qual seria o preço da venda?
- A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças? - - Qual o incremento no preço dado por cada opção de reforma?
</p>

<h2><font color='#252526'>Objetivos</font></h2>

<ul>
    <li> Predizer quais casas tem a maior probabilidade de lucro e o preço da casa.
    <li> Predizer quando vender as casas, e qual seria o preço 
    <li> Predizer se é necessário fazer reformas 
</ul>

<h2><font color='#734838'>Base de dados</font></h2>

Disponível em <a href='https://www.kaggle.com/datasets/shivachandel/kc-house-data'>Kaggle</a>

<h2><font color='#252526'>Ferramentas</font></h2>

  <img style='display:inline' src="https://img.shields.io/badge/Python-3.9.13-informational?style=for-the-badge&logo=python">
  
  <img style='display:inline' src="https://img.shields.io/badge/Jupyter-7.31.1-informational?style=for-the-badge&logo=jupyter">
  
   <img style='display:inline' src="https://img.shields.io/badge/Anaconda-22.9.0-informational?style=for-the-badge&logo=anaconda">

## 1.0 Coleta de Dados

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Bibliotecas necessárias para continuidade do projeto

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pandas_profiling import ProfileReport

%matplotlib inline

In [ ]:
# Localizando a base de dados na máquina
pwd = '/home/joel/Documentos/Dados/kc_house_data.csv'

# Instânciando um objeto dataframe com a base de dados
df = pd.read_csv(pwd)

In [ ]:
# Visualizando as 5 primeiras observações do dataset
df.head()

## 2.0 Limpeza de Dados

In [ ]:
# Visualizando dados gerais
df.info()

In [ ]:
# Visualizando dados nulos
df.isna().sum()

> Como há apenas 2 dados nulos em uma única variável, é razoável remover essas observações, uma vez que é irrelevante com a dimensão do dataset

In [ ]:
# Removendo observações com dados nulos
df.dropna(inplace=True)

In [ ]:
# Transformando a variável "date" no formato datetime
df['date'] = pd.to_datetime(df['date'])

## 3.0 Exploração de Dados

### 3.1 Exploração com ProfileReport

In [ ]:
# report = ProfileReport(df)

# report.to_file('profile_report.html')

### 3.2 Exploração com Seaborn

In [ ]:
# Plotando gráfico de correlação
axes = sns.heatmap(df.corr(), fmt='.1f', annot=True, cmap='coolwarm')
axes.figure.set_size_inches(10, 5)

In [ ]:
# Visualizando variáveis com maiores correlação com preço
(df.corr() > 0.5)['price']

In [ ]:
# Informações estatísticas sobre o dataset
df.describe()

In [ ]:
# Histograma da variavel de 'pés-quadrados'
df['sqft_living'].hist(bins = 50)

In [ ]:
# Instânciando dataframe com variaveis fortemente correlacionadas com preço
price_df = df[['price', 'date', 'bathrooms', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15']]

In [ ]:
# Separando dados em labels de acordo com pés-quadrados
price_df['label'] = pd.cut(price_df['sqft_living'],
       bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 13540],
       include_lowest=True,
       labels=['Muito pequena', 'Pequena', 'Média-Pequena', 'Média', 'Média-Grande', 'Grande', 'Muito Grande',])

In [ ]:
# Visualizando as 5 primeiras observações do novo dataset
price_df.head()

In [ ]:
price_df.groupby(by='label').describe()

In [ ]:
# Média de preço de cada tamanho de casa
price_df.groupby(by='label')['price'].mean()

## 4.0 Testando hipóteses

In [ ]:
# Extraindo todas as observações do tipo casa "média"
casas_media = price_df.query('label == "Média"')

In [ ]:
# Ordenando valores
casas_media = casas_media.sort_values(by='date')

In [ ]:
# Plotando gráficos
fig = px.line(casas_media, x='date', y='price', height=350)

fig.show()

In [ ]:
# Calculando a aceleração
casas_media['diff'] = casas_media['price'].diff()

In [ ]:
# Verificando se aumentou ou não
casas_media['increased'] = casas_media['diff'].apply(lambda x : x > 0)

In [ ]:
# Categorizando por dia
casas_media['day'] = casas_media['date'].dt.day

In [ ]:
# Aumento médio por dia
aumento_medio_por_dia = casas_media.groupby(by='day').mean()['increased']

In [ ]:
aumento_medio_por_dia = pd.DataFrame(aumento_medio_por_dia).reset_index()

In [ ]:
aumento_medio_por_dia['day'] = aumento_medio_por_dia['day'].apply(lambda x : 'd' + str(x))

In [ ]:
ax = sns.barplot(x = 'day', y = 'increased',
                 data = aumento_medio_por_dia.sort_values(by='increased', ascending=False),
                 palette='mako')

ax.figure.set_size_inches(12, 6)

## 5.0 Respondendo perguntas 

- Quais casas o CEO da House Rocket deveria comprar e por qual preço de compra?

> As casas do tipo pequena apresentam um menor risco, por ter um desvio padrão inferior aos demais, além de, em comparação lucro-risco, as casas do tipo pequena possuem melhores resultados, a média de um casa pequena é de 398.794,50 dólares, recomenda-se comprar, no dia 25 do mês, por apresentar uma tendência de maior baixa do valor

- Uma vez a casa em posse da empresa, qual o melhor momento para vendê-las e qual seria o preço da venda?

> O dia 31 do mês, apresenta em geral, um maior aumento no valor da casa, sendo assim, o melhor momento para vender, o preço em média é de 490.000,00 dólares


- A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças? Qual o incremento no preço dado por cada opção de reforma?

> As algumas variáveis que podem ser reformadas, apresentam correlação com preço da casa, então, provavelmente, reformas nesse setor aumentariam o valor da casa, apesar de que, para melhor cálculo, seria necessário os dados do valor da reforma